In [79]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import requests
import folium
from folium.plugins import HeatMap
import os
from sklearn.preprocessing import MinMaxScaler

print('libraries imported!')

libraries imported!


In [80]:
# Importing product_a.csv dataset into pandas DataFrame with first column as index

df_product_a = pd.read_csv('product_a.csv', index_col=0)
df_product_a.head()

,date_w,price,total_vol,plu1,plu2,plu3,bags_t,bags_s,bags_l,bags_lx,type,year,location
0,2016-12-24,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,A,2015,Albany
1,2016-12-17,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,A,2015,Albany
2,2016-12-10,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,A,2015,Albany
3,2016-12-03,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,A,2015,Albany
4,2016-11-26,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,A,2015,Albany


In [81]:
# Converting date_w field to a suitable datetime data type

# Alternative code - df_product_a.date_w.astype('datetime64[ns]')
df_product_a['date_w'] = pd.to_datetime(df_product_a.date_w, format='%Y-%m-%d')
df_product_a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18249 entries, 0 to 11
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date_w     18249 non-null  datetime64[ns]
 1   price      18249 non-null  float64       
 2   total_vol  18249 non-null  float64       
 3   plu1       18249 non-null  float64       
 4   plu2       18249 non-null  float64       
 5   plu3       18249 non-null  float64       
 6   bags_t     18249 non-null  float64       
 7   bags_s     18249 non-null  float64       
 8   bags_l     18249 non-null  float64       
 9   bags_lx    18249 non-null  float64       
 10  type       18249 non-null  object        
 11  year       18249 non-null  int64         
 12  location   18249 non-null  object        
dtypes: datetime64[ns](1), float64(9), int64(1), object(2)
memory usage: 1.9+ MB


In [82]:
# Correcting 'year' field values

df_product_a['year'] = df_product_a.date_w.dt.year
df_product_a.head()

,date_w,price,total_vol,plu1,plu2,plu3,bags_t,bags_s,bags_l,bags_lx,type,year,location
0,2016-12-24,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,A,2016,Albany
1,2016-12-17,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,A,2016,Albany
2,2016-12-10,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,A,2016,Albany
3,2016-12-03,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,A,2016,Albany
4,2016-11-26,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,A,2016,Albany


In [83]:
# Descriptive statistical analysis of 'df_product_a'

# numeric columns to use
columns_to_include = ['price', 'total_vol', 'plu1', 'plu2', 'plu3', 'bags_t', 'bags_s', 'bags_l', 'bags_lx']

df_stats = df_product_a.describe(percentiles=[0.1, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.9],
                                exclude=[np.object, np.int64, np.datetime64]).transpose()
# add columns - variance, median, mode
# NOTE: For mode, there is a tie between multiple values in 'total_vol' column, I am choosing one arbitrarily
df_stats['var'], df_stats['median'], df_stats['mode'] = df_product_a[columns_to_include].var(), df_product_a[columns_to_include].median(), df_product_a[columns_to_include].mode().iloc[0]
# adding IQR, skewness and kurtosis
df_stats['IQR'] = (df_stats['75%'] - df_stats['25%'])
df_stats['skewness'], df_stats['kurtosis'] = df_product_a.skew(), df_product_a.kurt()

# add 'field_name' column to 'df_stats'
df_stats.index.name = 'field_name'
df_stats.reset_index(inplace=True)

df_stats

,field_name,count,mean,std,min,10%,20%,25%,30%,40%,...,75%,80%,90%,max,var,median,mode,IQR,skewness,kurtosis
0,price,18249.0,1.405978,4.026766e-01,0.44,0.930,1.050,1.10,1.150,1.260,...,1.66,1.740,1.930,3.25,1.621484e-01,1.37,1.15,0.56,0.580303,0.325196
1,total_vol,18249.0,850644.013009,3.453545e+06,84.56,3896.768,8168.866,10838.58,15181.304,42137.088,...,432962.29,604868.968,1387045.760,62505646.52,1.192698e+13,107376.76,2038.99,422123.71,9.007687,92.104458
2,plu1,18249.0,293008.424531,1.264989e+06,0.00,94.276,483.362,854.07,1368.118,3261.580,...,111020.20,152679.068,538385.184,22743616.17,1.600197e+12,8645.30,0.00,110166.13,8.648220,86.809113
3,plu2,18249.0,295154.568356,1.204120e+06,0.00,367.484,1918.532,3008.78,4265.508,10961.994,...,150206.86,222163.620,500784.552,20470572.61,1.449906e+12,29061.02,0.00,147198.08,8.942466,91.949022
4,plu3,18249.0,22839.735993,1.074641e+05,0.00,0.000,0.000,0.00,0.000,42.704,...,6243.42,10972.600,31492.442,2546439.11,1.154853e+10,184.99,0.00,6243.42,10.159396,132.563441
5,bags_t,18249.0,239639.202060,9.862424e+05,0.00,1299.208,3347.674,5088.64,7316.634,16643.260,...,110783.37,149306.360,442141.928,19373134.37,9.726741e+11,39743.83,0.00,105694.73,9.756072,112.272156
6,bags_s,18249.0,182194.686696,7.461785e+05,0.00,583.110,1686.086,2849.42,4761.328,11502.076,...,83337.67,104537.112,354266.852,13384586.80,5.567824e+11,26362.82,0.00,80488.25,9.540660,107.012885
7,bags_l,18249.0,54338.088145,2.439660e+05,0.00,0.000,30.918,127.47,329.944,1105.110,...,22029.25,34350.986,94295.338,5719096.61,5.951939e+10,2647.71,0.00,21901.78,9.796455,117.999481
8,bags_lx,18249.0,3106.426507,1.769289e+04,0.00,0.000,0.000,0.00,0.000,0.000,...,132.50,557.114,3688.912,551693.65,3.130385e+08,0.00,0.00,132.50,13.139751,233.602612


#### Discussion
##### Part a
The mathematical theory/relationship that lies behind some of these statistical measures are as follows:
1. 25th percentile = 1st quartile
2. Median = 50th percentile = 2nd quartile
3. 75th percentile = 3rd quartile
4. Variance = (Standard deviation)^2
5. Mean = (Total sum) / Count
6. IQR = (3rd quartile - 1st quartile) = (75th percentile - 25th percentile)
7. Range = max - min
    

##### Part b
Usage of 'df_stats' columns in Data Analysis of a dataset are as follows:
1. Columns - 'count', 'min' and 'max' summarises the total observations, minimum and maximum value in a target column data.
2. Columns - 'mean' and 'standard deviation' are used to characterize the centre/location (mean) and variability (std) of the target data.
3. Columns - '10%'...'90%' are percentile values that depict the value below which a given % of observations fall. 
4. Column - 'IQR' is another measure of spread in the dataset, which measures the midspread (middle 50% data) range in a dataset.
5. Columns - 'skewness' and 'kurtosis' are used mainly to verify if the distribution of the target column is near to a normal distribution. Specifically, skewness measures the amount of symmetry observed in the target column data, whereas kurtosis evaluates how heavy target column's tails are relative to a normal distribution.
    
##### Part c
Descriptive statistical analysis of 'df_product_a':
1. All fields have outliers in their data that are far far away from the rest - as shown by large difference between '90th percentile' and 'max' values. For example, 'max' value in 'plu3' column is 2546439.11, while 90% of the data is below 31492.442.
2. 'mean' and 'var' values of the fields of 'df_product_a' are very very large due to the presence of outliers.
3. None of the fields have 'skewness' or 'kurtosis' values close to 0, meaning none of them are normally distributed. Field 'price' is the most close to a normal distribution.
4. There are no missing values in the dataset.

In [84]:
# Creating a Pearson correlation matrix

df_coef_p = df_product_a.drop(['year'], axis=1).corr()
df_coef_p.style.background_gradient(cmap='coolwarm').set_precision(2)

,price,total_vol,plu1,plu2,plu3,bags_t,bags_s,bags_l,bags_lx
price,1.00,-0.19,-0.21,-0.17,-0.18,-0.18,-0.17,-0.17,-0.12
total_vol,-0.19,1.00,0.98,0.97,0.87,0.96,0.97,0.88,0.75
plu1,-0.21,0.98,1.00,0.93,0.83,0.92,0.93,0.84,0.70
plu2,-0.17,0.97,0.93,1.00,0.89,0.91,0.92,0.81,0.69
plu3,-0.18,0.87,0.83,0.89,1.00,0.79,0.80,0.70,0.68
bags_t,-0.18,0.96,0.92,0.91,0.79,1.00,0.99,0.94,0.80
bags_s,-0.17,0.97,0.93,0.92,0.80,0.99,1.00,0.90,0.81
bags_l,-0.17,0.88,0.84,0.81,0.70,0.94,0.90,1.00,0.71
bags_lx,-0.12,0.75,0.70,0.69,0.68,0.80,0.81,0.71,1.00


In [85]:
# Creating a Spearman Rank correlation matrix

df_coef_sp = df_product_a.corr(method='spearman')
df_coef_sp.style.background_gradient(cmap='coolwarm').set_precision(2)

,price,total_vol,plu1,plu2,plu3,bags_t,bags_s,bags_l,bags_lx,year
price,1.00,-0.61,-0.60,-0.51,-0.53,-0.60,-0.54,-0.51,-0.43,0.11
total_vol,-0.61,1.00,0.90,0.94,0.83,0.95,0.93,0.70,0.66,0.09
plu1,-0.60,0.90,1.00,0.79,0.75,0.84,0.82,0.64,0.62,0.01
plu2,-0.51,0.94,0.79,1.00,0.82,0.86,0.83,0.64,0.63,0.01
plu3,-0.53,0.83,0.75,0.82,1.00,0.78,0.78,0.57,0.65,-0.04
bags_t,-0.60,0.95,0.84,0.86,0.78,1.00,0.96,0.75,0.65,0.21
bags_s,-0.54,0.93,0.82,0.83,0.78,0.96,1.00,0.60,0.64,0.18
bags_l,-0.51,0.70,0.64,0.64,0.57,0.75,0.60,1.00,0.50,0.19
bags_lx,-0.43,0.66,0.62,0.63,0.65,0.65,0.64,0.50,1.00,0.17
year,0.11,0.09,0.01,0.01,-0.04,0.21,0.18,0.19,0.17,1.00


In [86]:
# Exploring relationships between 'total_vol' and other fields of 'product_a' dataset

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

lr = LinearRegression()
X = df_product_a.loc[:, ['price', 'plu1', 'plu2', 'plu3', 'bags_t', 'bags_s', 'bags_l', 'bags_lx', 'total_vol']]

X_train, X_test, y_train, y_test = train_test_split(X.iloc[:, :-1], X['total_vol'], test_size=0.33)

lr.fit(X_train, y_train)
print('Accuracy on test set: ', lr.score(X_test, y_test)*100)

# important findings
print('Co-efficients of fields wrt \'total_vol\':')
for idx, value in enumerate(lr.coef_):
    print('\t', X.columns[idx], ': ', round(value, 2))

# relationship found:  total_vol = constant + plu1 + plu2 + plu3 + 0.95*bags_t

Accuracy on test set:  99.9999999964687
Co-efficients of fields wrt 'total_vol':
	 price :  3.29
	 plu1 :  1.0
	 plu2 :  1.0
	 plu3 :  1.0
	 bags_t :  1.74
	 bags_s :  -0.74
	 bags_l :  -0.74
	 bags_lx :  -0.74


#### DIscussion
1. Column 'total_vol' is demonstrating a strong positive correlation with columns - 'plu1', 'plu2' and 'plu3' as seen from both the correlation plots above. To further investigate the relationship, I have also trained a Linear Regression model with 'total_vol' as my dependent variable. The results are in-line with the correlation plots obtained after training, we can see that the co-efficients learned by the model for 'plu1', 'plu2' and 'plu3' independent varaibles is 1 i.e. showing a strong positive relationship with 'total_vol'.

2. In addition, columns 'bags_t' and 'bags_s' are having a very strong positive correlation (r=0.99 and rho=0.96), which is also evidenced in the scatter plots below.

3. 'price' column is showing a negative correlation with other fields in both the plots, which can denote the fact that price decreases with the increase in quantity of goods/service.

In [87]:
# Create a Plotly scatter matrix plot for df_product_a

# What is 'trace' in plotly? 
# A trace is just the name we give a collection of data and the specifications of which we want that data plotted

fig = px.scatter_matrix(df_product_a.drop(['date_w'], axis=1), 
                        width=1500,
                        height=1500,
                        opacity=0.4, 
                        title='Scatter matrix plot for <i>df_product_a</i>'
                       )
fig.update_layout(title_x=0.5)
# open the figure in another tab as it is large in size
plot(fig, filename='scatter_matrix_plot.html')

'scatter_matrix_plot.html'

In [88]:
# Creating weekly and monthly time-series graphs of the numeric fields

# convert the dataframe into a time-series format
df_product_a_ts = df_product_a.drop(['type', 'year', 'location'], axis=1).set_index('date_w').sort_index()

# downsample the data to weekly data points using average values for each week (weekly)
df_product_a_weekly = df_product_a_ts.resample('W').mean().dropna()
# downsample the data to monthly data points using average values for each month (monthly)
df_product_a_monthly = df_product_a_ts.resample('M').mean().dropna()

# creating traces
fig = make_subplots(rows=2, cols=1, specs=[[{"secondary_y": True}], [{"secondary_y": True}]])

# colors to use
colors = px.colors.cyclical.HSV[0:7]

# columns 'price' and 'total_vol' are not included as there scales are too small or large (compared to others)
upper_cols = ['plu1', 'plu2', 'bags_t', 'bags_s', 'bags_l'] # upper range columns (in 10^5s)
lower_cols = ['plu3', 'bags_lx'] # lower range columns (in 10^4s)

# upper columns
for col, color in zip(upper_cols, colors[0:5]):
    # weekly
    fig.add_trace(go.Scatter(x=df_product_a_weekly.index, y=df_product_a_weekly[col],
                        mode='lines+markers', line=dict(color=color),
                        name='{} weekly'.format(col)), row=1, col=1, secondary_y=False)
    # reducing marker size
    fig.update_traces(marker=dict(size=4), row=1, col=1)
    
    # monthly
    fig.add_trace(go.Scatter(x=df_product_a_monthly.index, y=df_product_a_monthly[col],
                        mode='lines+markers', line=dict(color=color),
                        name='{} monthly'.format(col)), row=2, col=1, secondary_y=False)

# lower columns
for col, color in zip(lower_cols, colors[5:7]):
    # weekly
    fig.add_trace(go.Scatter(x=df_product_a_weekly.index, y=df_product_a_weekly[col],
                        mode='lines+markers', line=dict(color=color),
                        name='{} weekly'.format(col)), row=1, col=1, secondary_y=True)
    
    # reducing marker size
    fig.update_traces(marker=dict(size=4), row=1, col=1)
    
    # monthly
    fig.add_trace(go.Scatter(x=df_product_a_monthly.index, y=df_product_a_monthly[col],
                        mode='lines+markers', line=dict(color=color),
                        name='{} monthly'.format(col)), row=2, col=1, secondary_y=True)
    

fig.update_layout(height=1000, title_text='Weekly and monthly time-series graphs', title_x=0.5)
# setting x-axis title
fig.update_xaxes(title_text='Week', row=1, col=1)
fig.update_xaxes(title_text='Month', row=2, col=1)
# setting y-axis title
fig.update_yaxes(title_text='Field values')

plot(fig, filename='weekly_monthly_timeseries_graphs.html')
#iplot(fig)

'weekly_monthly_timeseries_graphs.html'

#### Discussion
Major insights of Weekly graph:
There was a huge short term spike in all the numeric fields, especially in 'plu1','pu2' and 'plu3', in the last week of month January, 2016. Similarly, a huge spike was seen in the first week of February, 2017. Similar trend followed for the next two years i.e. 2018 and 2019 as well, indicating a strong pattern of short term hikes in values of the numerical fields in the month of February.

Major insights of Monthly graph:
A full cycle comprising of upward and downward trends can be observed in values of the numerical fields, which is a typical trend when performing long term analyses.

In [89]:
# Creating year based location and type bar charts for the total volume

# years to analyse
years = [2016, 2017, 2018, 2019]

# 4 subplots - one for each year
fig = make_subplots(rows=4, cols=1, vertical_spacing=0.12, subplot_titles=('2016', '2017', '2018', '2019'))

for idx, year in enumerate(years):
    # type 'A'
    df_product_a_A = df_product_a.loc[((df_product_a.year == year) & (df_product_a.type == 'A')), ['total_vol', 'location', 'type']]
    # grouped by locations
    _A = df_product_a_A.groupby('location')['total_vol'].sum()
    # dropping 'TotalUS' to avoid scaling issues
    _A.drop('TotalUS', inplace=True)
    # type 'C'
    df_product_a_C = df_product_a.loc[((df_product_a.year == year) & (df_product_a.type == 'C')), ['total_vol', 'location', 'type']]
    # grouped by locations
    _C = df_product_a_C.groupby('location')['total_vol'].sum()
    # dropping 'TotalUS' to avoid scaling issues
    _C.drop('TotalUS', inplace=True)
    
    fig.add_trace(go.Bar(name=f'Type A - {year}', x=_A.index, y=_A), idx + 1, 1)
    fig.add_trace(go.Bar(name=f'Type C - {year}', x=_C.index, y=_C), idx + 1, 1)
    
    # labelling y-axis
    fig.update_yaxes(title_text='Total count of <i>total_vol</i>', row=idx + 1, col=1)


# plotting grouped bar chart
fig.update_layout(barmode='group', height=1500)    

plot(fig, filename='total_vol_by_location_type.html')

'total_vol_by_location_type.html'

#### Discussion
1. As visualised in the charts, the sum of 'total_vol' field in type 'C'  is almost negligible relative to type 'A' in any location.
2. 'Northeast', 'California', 'West' and 'SouthCentral' are successful in maintaining their positions in top 4 spot for type 'A' for at least 4 consecutive years (from 2016 - 2019).
3. The sum of 'total_vol' has dropped significantly (by two to three folds) for all the locations in the year 2019 as compared to previous years. ('price' also drops in 2019 as evidenced in the scatter plot - not sure if these two incidents are related though).

In [90]:
# Geolocations API service credentials

GOOGLE_API_KEY = 'Your key'
HERE_API_KEY = 'Your key'

In [91]:
# Functions to retrieve geo coordinates (i.e. longitudes and latitudes) - Google, Here and ArcGic API services

def get_geo_coordinates_from_google(address:str, connection_params:dict):
    base_url = 'https://maps.googleapis.com/maps/api/geocode'
    url = '{}/{}?address={}&key={}'.format(base_url,
                          connection_params['output_format'],
                          address,
                          connection_params['api_key']    
                         )
    
    # make the GET request
    results = requests.get(url).json()
    #print(address, results)
    
    # check if codes were successfully obtained or not
    if results['status'] == 'ZERO_RESULTS':
        return None
    
    location = results['results'][0]['geometry']['location']
    return {
        'longitude' : location['lng'],
        'latitude' : location['lat']
    }

def get_geo_coordinates_from_here(address:str, connection_params:dict):
    base_url = 'https://geocoder.ls.hereapi.com/6.2/geocode.'
    url = '{}{}?searchtext={}&gen=9&apiKey={}'.format(base_url,
                                                      connection_params['output_format'],
                                                      address, connection_params['api_key']
                                                     )
    # make the GET request
    results = requests.get(url).json()
    
    # check if codes were successfully obtained or not
    if len(results['Response']['View']) == 0:
        return None
    
    location = results['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']
    return {
        'longitude': location['Longitude'],
        'latitude': location['Latitude']
    }

def get_geo_coordinates_from_arcgis(address:str, connection_params:dict):
    base_url = 'https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates'
    url = f'{base_url}?f={connection_params["output_format"]}&singleLine={address}&outFields={connection_params["out_fields"]}'
    results = requests.get(url).json()

    location = results['candidates'][0]['location']
    return {
        'longitude': location['x'],
        'latitude': location['y']
    }


# testing functions
# Google API parameters
connection_params = {
    'output_format': 'json',
    'api_key': GOOGLE_API_KEY
}
address = 'Boise,+US'

lng_lat_google = get_geo_coordinates_from_google(address, connection_params)

# Here API paramters
connection_params = {
    'output_format': 'json',
    'api_key': HERE_API_KEY
}
address = 'Boise,+US'

lng_lat_here = get_geo_coordinates_from_here(address, connection_params)

# ArcGis parameters
connection_params = {
    'output_format': 'json',
    'out_fields': 'Match_addr,Addr_type'
}
address = 'Boise, US'

lng_lat_arcgis = get_geo_coordinates_from_arcgis(address, connection_params)

# printing
lng_lat_google, lng_lat_here, lng_lat_arcgis

({'longitude': -116.2023137, 'latitude': 43.6150186},
 {'longitude': -116.19341, 'latitude': 43.60765},
 {'longitude': -116.19339999999994, 'latitude': 43.60764000000006})

In [92]:
# Getting geocodes for all the unique location values in 'product_a' dataset

connection_params = {
    'output_format': 'json',
    'api_key': GOOGLE_API_KEY
}

location_geocodes_google = {addr: get_geo_coordinates_from_google('{},+US'.format(addr), connection_params) for addr in df_product_a['location'].unique()}

connection_params = {
    'output_format': 'json',
    'api_key': HERE_API_KEY
}

location_geocodes_here = {addr: get_geo_coordinates_from_here('{},+US'.format(addr), connection_params) for addr in df_product_a['location'].unique()}

connection_params = {
    'output_format': 'json',
    'out_fields': 'Match_addr,Addr_type'
}

location_geocodes_arcgis = {addr: get_geo_coordinates_from_arcgis('{},+US'.format(addr), connection_params) for addr in df_product_a['location'].unique()}

In [93]:
# Visualising location data with Folium map from all the three Geocoding service providers

# USA coordinates
latitude = 37.2754919
longitude = -104.6582933
# create map of United States using latitude and longitude values
map_us = folium.Map(location=[latitude, longitude], zoom_start=3)
data = []

# add markers to map
for address in location_geocodes_arcgis.keys():
    # check if location geocodes are present
    if location_geocodes_arcgis[address] is not None:
        lat = location_geocodes_arcgis[address]['latitude']
        lng = location_geocodes_arcgis[address]['longitude']

        address = address + ', US'
        label = folium.Popup(address, parse_html=True)
        # add to the map
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=folium.Popup(address + ' - ArcGis'),
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_us)
    else:
        print('"{}" geocodes not found'. format(address+', US'))

# add markers to map - Google
for address in location_geocodes_google.keys():
    # check if location geocodes are present
    if location_geocodes_google[address] is not None:
        lat = location_geocodes_google[address]['latitude']
        lng = location_geocodes_google[address]['longitude']

        address = address + ', US'
        label = folium.Popup(address, parse_html=True)
        # add to the map
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=folium.Popup(address + ' - Google'),
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_us)
    else:
        print('"{}" geocodes not found (Google)'. format(address+', US'))

# add markers to map - Here
for address in location_geocodes_here.keys():
    # check if location geocodes are present
    if location_geocodes_here[address] is not None:
        lat = location_geocodes_here[address]['latitude']
        lng = location_geocodes_here[address]['longitude']

        address = address + ', US'
        label = folium.Popup(address, parse_html=True)
        # add to the map
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=folium.Popup(address + ' - Here'),
            color='green',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_us)
    else:
        print('"{}" geocodes not found (Here)'. format(address+', US'))


# average values of bags_t per location
mean_bags_t = df_product_a.groupby('location')['bags_t'].mean()
data = dict(location_geocodes)

# remove null values
if 'TotalUS' in data:
    del data['TotalUS']
if 'WestTexNewMexico' in data:
    del data['WestTexNewMexico']

# get 'weight' (i.e. mean bags_t value) for each location
for key in data.keys():
    data[key]['weight'] = mean_bags_t.loc[mean_bags_t.index == key].values[0]
    
# convert it to list of lists for HeatMap method
data = pd.DataFrame(data).T
heat_data = [[row['latitude'],row['longitude'], row['weight']] for index, row in data.iterrows()]

#map_us.add_child(HeatMap(data[:, 0:2], radius=15))
#HeatMap(heat_data, radius=15).add_to(map_us)

map_us

"TotalUS, US" geocodes not found (Google)
"WestTexNewMexico, US" geocodes not found (Google)
"BaltimoreWashington, US" geocodes not found (Here)
"BuffaloRochester, US" geocodes not found (Here)
"CincinnatiDayton, US" geocodes not found (Here)
"DallasFtWorth, US" geocodes not found (Here)
"HarrisburgScranton, US" geocodes not found (Here)
"Midsouth, US" geocodes not found (Here)
"NewOrleansMobile, US" geocodes not found (Here)
"PhoenixTucson, US" geocodes not found (Here)
"RaleighGreensboro, US" geocodes not found (Here)
"RichmondNorfolk, US" geocodes not found (Here)
"SouthCarolina, US" geocodes not found (Here)
"TotalUS, US" geocodes not found (Here)


In [ ]:
# Using Google Geocoding API service for further tasks
location_geocodes = location_geocodes_google.copy()

#### Discusion
In terms of accuracy, Google API failed to find geocodes for 2 locations, HERE API had a count of 12 failures and ArcGis provided geocodes for all the locations, however, the integrity of the data returned can be questionable as one of the locations is landing on inslands in Pacific ocean i.e. far outside the US borders.

In terms of latency, Google API and HERE API outperformed ARCGIS API which took the most ammount of time for geocoding the 54 unique locations.

Considering all of the factors discussed above, I will be using Google API for further tasks below.

In [ ]:
# Using Google Reverse Geocoding API service

def get_state_from_lat_lng(lat, lng):
    lat_lng = '{},{}'.format(lat, lng)
    url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={}&key={}'.format(
        lat_lng,
        GOOGLE_API_KEY
    )
    # make GET request to end-point
    results = requests.get(url).json()
    
    for dict_ in results['results'][0]['address_components']:
        if dict_['types'] == ['administrative_area_level_1', 'political']:
            return dict_['short_name']

In [ ]:
# Data used by Folium map

# data - total bags_t
# total values of bags_t per location
total_bags_t = df_product_a.groupby('location')['bags_t'].sum()
# total values of bags_t per location and type
total_bags_t_type = df_product_a.groupby(['location', 'type'])['bags_t'].sum()

# data - mean bags_t
mean_bags_t = df_product_a.groupby('location')[['bags_t']].mean()
mean_bags_t.drop(['TotalUS', 'WestTexNewMexico'], inplace=True)
mean_bags_t['binned color'] = pd.cut(mean_bags_t['bags_t'], bins=3, labels=['pink', 'orange', 'darkred'])
# add 'lat' and 'lon' values
mean_bags_t['latitude'], mean_bags_t['longitude'] = mean_bags_t.apply(lambda row: location_geocodes[row.name]['latitude'], axis=1), mean_bags_t.apply(lambda row: location_geocodes[row.name]['longitude'], axis=1)
# add corresponding state_names
mean_bags_t['state name'] = mean_bags_t.apply(lambda row: get_state_from_lat_lng(row['latitude'], row['longitude']), axis=1)
state_mean_bags_t = mean_bags_t[['state name', 'bags_t']].reset_index(drop=True)
# get average bags_t values per state
state_mean_bags_t = mean_bags_t.groupby('state name')[['bags_t']].mean().reset_index()

# data - mean values of all numerical fields
mean_data = df_product_a.groupby('location').mean()
mean_data = mean_data.drop('year', axis=1)

# data - mean price of type A per location
a = df_product_a[df_product_a.type == 'A']
mean_price_a = df_product_a.groupby('location')[['price']].mean()

In [ ]:
# Visualising data with Folium map

# USA coordinates
latitude = 37.2754919
longitude = -104.6582933

# create map of United States using latitude and longitude values
map_us = folium.Map(location=[latitude, longitude], zoom_start=4)

# choropleth map geoJSON file
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_mean_bags_t,
    columns=['state name', 'bags_t'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Mean bags_t values'
).add_to(map_us)

folium.LayerControl().add_to(map_us)

# add markers to map
for address in location_geocodes.keys():
    # check if location geocodes are present
    if location_geocodes[address] is not None:
        lat = location_geocodes[address]['latitude']
        lng = location_geocodes[address]['longitude']

        tooltip = '{}<br>Total bags_t: {}<br>Total bags_t (Type \'A\'): {}<br>Total bags_t (Type \'C\'): {}'.format(
            address + ', US',
            round(total_bags_t.loc[total_bags_t.index == address].values[0], 2),
            round(total_bags_t_type.loc[(address)]['A'], 2),
            round(total_bags_t_type.loc[(address)]['C'], 2)
        )
                
        label = 'Mean values:<br>'
        for index, value in mean_data.loc[address].iteritems():
            label += '{}: {}<br>'.format(index, round(value, 2))
        
        # add to the map
        folium.CircleMarker(
            [lat, lng],
            radius=mean_price_a.loc[address].values[0]*5,
            tooltip=folium.Tooltip(tooltip, style='bold', sticky=False),
            popup=folium.Popup(label, max_width=150),
            color='#000000',
            fill=True,
            fill_color=mean_bags_t.loc[address, 'binned color'],
            fill_opacity=0.7,
            parse_html=False).add_to(map_us)
    else:
        print('"{}" geocodes not found :('. format(address+', US'))

map_us

#### Discusion
In terms of accuracy, Google API failed to find geocodes for 2 locations, HERE API had a count of 12 failures and ArcGis provided geocodes for all the locations, however, the integrity of the data returned can be questionable as one of the locations is landing on inslands in Pacific ocean i.e. far outside the US borders.

In terms of latency, Google API and HERE API outperformed ARCGIS API which took the most ammount of time for geocoding the 54 unique locations.

Considering all of the factors discussed above, I will be using Google API for further tasks below.

In [ ]:
# Using Google Reverse Geocoding API service

def get_state_from_lat_lng(lat, lng):
    lat_lng = '{},{}'.format(lat, lng)
    url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={}&key={}'.format(
        lat_lng,
        GOOGLE_API_KEY
    )
    # make GET request to end-point
    results = requests.get(url).json()
    
    for dict_ in results['results'][0]['address_components']:
        if dict_['types'] == ['administrative_area_level_1', 'political']:
            return dict_['short_name']

In [ ]:
# Data used by Folium map

# data - total bags_t
# total values of bags_t per location
total_bags_t = df_product_a.groupby('location')['bags_t'].sum()
# total values of bags_t per location and type
total_bags_t_type = df_product_a.groupby(['location', 'type'])['bags_t'].sum()

# data - mean bags_t
mean_bags_t = df_product_a.groupby('location')[['bags_t']].mean()
mean_bags_t.drop(['TotalUS', 'WestTexNewMexico'], inplace=True)
mean_bags_t['binned color'] = pd.cut(mean_bags_t['bags_t'], bins=3, labels=['pink', 'orange', 'darkred'])
# add 'lat' and 'lon' values
mean_bags_t['latitude'], mean_bags_t['longitude'] = mean_bags_t.apply(lambda row: location_geocodes[row.name]['latitude'], axis=1), mean_bags_t.apply(lambda row: location_geocodes[row.name]['longitude'], axis=1)
# add corresponding state_names
mean_bags_t['state name'] = mean_bags_t.apply(lambda row: get_state_from_lat_lng(row['latitude'], row['longitude']), axis=1)
state_mean_bags_t = mean_bags_t[['state name', 'bags_t']].reset_index(drop=True)
# get average bags_t values per state
state_mean_bags_t = mean_bags_t.groupby('state name')[['bags_t']].mean().reset_index()

# data - mean values of all numerical fields
mean_data = df_product_a.groupby('location').mean()
mean_data = mean_data.drop('year', axis=1)

# data - mean price of type A per location
a = df_product_a[df_product_a.type == 'A']
mean_price_a = df_product_a.groupby('location')[['price']].mean()

In [ ]:
# Visualising data with Folium map

# USA coordinates
latitude = 37.2754919
longitude = -104.6582933

# create map of United States using latitude and longitude values
map_us = folium.Map(location=[latitude, longitude], zoom_start=4)

# choropleth map geoJSON file
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_mean_bags_t,
    columns=['state name', 'bags_t'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Mean bags_t values'
).add_to(map_us)

folium.LayerControl().add_to(map_us)

# add markers to map
for address in location_geocodes.keys():
    # check if location geocodes are present
    if location_geocodes[address] is not None:
        lat = location_geocodes[address]['latitude']
        lng = location_geocodes[address]['longitude']

        tooltip = '{}<br>Total bags_t: {}<br>Total bags_t (Type \'A\'): {}<br>Total bags_t (Type \'C\'): {}'.format(
            address + ', US',
            round(total_bags_t.loc[total_bags_t.index == address].values[0], 2),
            round(total_bags_t_type.loc[(address)]['A'], 2),
            round(total_bags_t_type.loc[(address)]['C'], 2)
        )
                
        label = 'Mean values:<br>'
        for index, value in mean_data.loc[address].iteritems():
            label += '{}: {}<br>'.format(index, round(value, 2))
        
        # add to the map
        folium.CircleMarker(
            [lat, lng],
            radius=mean_price_a.loc[address].values[0]*5,
            tooltip=folium.Tooltip(tooltip, style='bold', sticky=False),
            popup=folium.Popup(label, max_width=150),
            color='#000000',
            fill=True,
            fill_color=mean_bags_t.loc[address, 'binned color'],
            fill_opacity=0.7,
            parse_html=False).add_to(map_us)
    else:
        print('"{}" geocodes not found :('. format(address+', US'))

map_us

#### Discusion
In terms of accuracy, Google API failed to find geocodes for 2 locations, HERE API had a count of 12 failures and ArcGis provided geocodes for all the locations, however, the integrity of the data returned can be questionable as one of the locations is landing on inslands in Pacific ocean i.e. far outside the US borders.

In terms of latency, Google API and HERE API outperformed ARCGIS API which took the most ammount of time for geocoding the 54 unique locations.

Considering all of the factors discussed above, I will be using Google API for further tasks below.

In [95]:
# Using Google Reverse Geocoding API service

def get_state_from_lat_lng(lat, lng):
    lat_lng = '{},{}'.format(lat, lng)
    url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={}&key={}'.format(
        lat_lng,
        GOOGLE_API_KEY
    )
    # make GET request to end-point
    results = requests.get(url).json()
    
    for dict_ in results['results'][0]['address_components']:
        if dict_['types'] == ['administrative_area_level_1', 'political']:
            return dict_['short_name']

In [96]:
# Data used by Folium map

# data - total bags_t
# total values of bags_t per location
total_bags_t = df_product_a.groupby('location')['bags_t'].sum()
# total values of bags_t per location and type
total_bags_t_type = df_product_a.groupby(['location', 'type'])['bags_t'].sum()

# data - mean bags_t
mean_bags_t = df_product_a.groupby('location')[['bags_t']].mean()
mean_bags_t.drop(['TotalUS', 'WestTexNewMexico'], inplace=True)
mean_bags_t['binned color'] = pd.cut(mean_bags_t['bags_t'], bins=3, labels=['pink', 'orange', 'darkred'])
# add 'lat' and 'lon' values
mean_bags_t['latitude'], mean_bags_t['longitude'] = mean_bags_t.apply(lambda row: location_geocodes[row.name]['latitude'], axis=1), mean_bags_t.apply(lambda row: location_geocodes[row.name]['longitude'], axis=1)
# add corresponding state_names
mean_bags_t['state name'] = mean_bags_t.apply(lambda row: get_state_from_lat_lng(row['latitude'], row['longitude']), axis=1)
state_mean_bags_t = mean_bags_t[['state name', 'bags_t']].reset_index(drop=True)
# get average bags_t values per state
state_mean_bags_t = mean_bags_t.groupby('state name')[['bags_t']].mean().reset_index()

# data - mean values of all numerical fields
mean_data = df_product_a.groupby('location').mean()
mean_data = mean_data.drop('year', axis=1)

# data - mean price of type A per location
a = df_product_a[df_product_a.type == 'A']
mean_price_a = df_product_a.groupby('location')[['price']].mean()

In [97]:
# Visualising data with Folium map

# USA coordinates
latitude = 37.2754919
longitude = -104.6582933

# create map of United States using latitude and longitude values
map_us = folium.Map(location=[latitude, longitude], zoom_start=4)

# choropleth map geoJSON file
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_mean_bags_t,
    columns=['state name', 'bags_t'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Mean bags_t values'
).add_to(map_us)

folium.LayerControl().add_to(map_us)

# add markers to map
for address in location_geocodes.keys():
    # check if location geocodes are present
    if location_geocodes[address] is not None:
        lat = location_geocodes[address]['latitude']
        lng = location_geocodes[address]['longitude']

        tooltip = '{}<br>Total bags_t: {}<br>Total bags_t (Type \'A\'): {}<br>Total bags_t (Type \'C\'): {}'.format(
            address + ', US',
            round(total_bags_t.loc[total_bags_t.index == address].values[0], 2),
            round(total_bags_t_type.loc[(address)]['A'], 2),
            round(total_bags_t_type.loc[(address)]['C'], 2)
        )
                
        label = 'Mean values:<br>'
        for index, value in mean_data.loc[address].iteritems():
            label += '{}: {}<br>'.format(index, round(value, 2))
        
        # add to the map
        folium.CircleMarker(
            [lat, lng],
            radius=mean_price_a.loc[address].values[0]*5,
            tooltip=folium.Tooltip(tooltip, style='bold', sticky=False),
            popup=folium.Popup(label, max_width=150),
            color='#000000',
            fill=True,
            fill_color=mean_bags_t.loc[address, 'binned color'],
            fill_opacity=0.7,
            parse_html=False).add_to(map_us)
    else:
        print('"{}" geocodes not found :('. format(address+', US'))

map_us

"TotalUS, US" geocodes not found :(
"WestTexNewMexico, US" geocodes not found :(
